This notebook shows how to use the general interface in HiGP for regression problems.

In this test, we will use the 3D Road dataset at "https://archive.ics.uci.edu/ml/machine-learning-databases/00246/3D_spatial_network.txt". 

We random sample 30000 points for training and use 100 points for testing for demonstration.

In [1]:
import higp
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import torch
%matplotlib inline

Since this is a large (> 20000 training points) and low-dimensional (2D or 3D) data set, HiGP can use the $\mathcal{H}^2$ matrix for faster calculation. The $\mathcal{H}^2$ matrix requires a higher working precision, so we use float64 instead of float32 in this example.

In [2]:
np_dtype = np.float64

Download the dataset, Scale features to $[-1, 1]$ and normalize labels.

In [3]:
df = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/00246/3D_spatial_network.txt", sep=',', header=None)
data_array = df.values[:, 1:]
all_x = data_array[:, 1:3]
all_y = data_array[:, -1]

# Scale features and normalize labels
all_x_max = np.max(all_x, 0)
all_x_min = np.min(all_x, 0)
all_x = 2.0 * (all_x - all_x_min[np.newaxis, :]) / (all_x_max[np.newaxis, :] - all_x_min[np.newaxis, :]) - 1.0
all_y = (all_y - np.mean(all_y)) / np.std(all_y)

# Randomly select a subset of the data
n_all = all_x.shape[0]
n_train = 30000
n_test = 100
n_sample = n_train + n_test

sample_array = np.random.choice(n_all, n_sample, replace = False)

train_x = np.ascontiguousarray(all_x[sample_array[:n_train], :].T).astype(np_dtype)
train_y = np.ascontiguousarray(all_y[sample_array[:n_train]]).astype(np_dtype)
test_x = np.ascontiguousarray(all_x[sample_array[n_train:], :].T).astype(np_dtype)
test_y = np.ascontiguousarray(all_y[sample_array[n_train:]]).astype(np_dtype)

Remember to use the `ascontiguousarray()` method in NumPy to guarantee that `train_x, train_y, test_x, test_y` are stored contiguously.

Now let's check the shapes of these four arrays. We can see that each data point is stored in one column in `train_x` and `test_x`.

In [4]:
print("Shape of train_x : ", train_x.shape)
print("Shape of train_y : ", train_y.shape)
print("Shape of test_x : ", test_x.shape)
print("Shape of test_y : ", test_y.shape)

Shape of train_x :  (2, 30000)
Shape of train_y :  (30000,)
Shape of test_x :  (2, 100)
Shape of test_y :  (100,)


Create a GP regression problem model and a PyTorch Adam optimizer. 

By default, HiGP uses the $\mathcal{H}^2$ matrix if possible (`mvtype = higp.MatvecAuto`). If you want to disable the use of the $\mathcal{H}^2$ matrix, set `mvtype = higp.MatvecAOT`. For more information about the parameter `mvtype`, please refer to the user manual. 

In [5]:
torch_dtype = torch.float32 if np_dtype == np.float32 else torch.float64
gprproblem = higp.gprproblem.setup(data=train_x, label=train_y, kernel_type=higp.GaussianKernel, mvtype=higp.MatvecAuto)
model = higp.GPRModel(gprproblem, dtype=torch_dtype)
optimizer = torch.optim.Adam(model.parameters(), lr=0.05)

Run 20 steps of Adam.

In [6]:
loss_history, param_histpry = higp.gpr_torch_minimize(model, optimizer, maxits=20, print_info=True)

Iteration (max 20), Elapsed time (sec), Loss, Hyperparameters (l, s, f, before nnt)
1, 15.63, 0.73830, 0.050, -0.050, 0.050
2, 22.54, 0.72007, 0.100, -0.086, 0.100
3, 29.37, 0.70694, 0.149, -0.127, 0.150
4, 36.32, 0.69174, 0.198, -0.171, 0.199
5, 43.33, 0.67514, 0.245, -0.210, 0.248
6, 50.18, 0.66023, 0.292, -0.249, 0.297
7, 57.12, 0.64546, 0.338, -0.291, 0.345
8, 64.09, 0.62957, 0.382, -0.334, 0.393
9, 71.00, 0.61284, 0.424, -0.377, 0.441
10, 78.03, 0.59606, 0.466, -0.421, 0.488
11, 84.79, 0.57911, 0.505, -0.464, 0.534
12, 91.67, 0.56199, 0.543, -0.506, 0.580
13, 98.58, 0.54539, 0.579, -0.548, 0.625
14, 105.45, 0.52824, 0.614, -0.590, 0.670
15, 112.38, 0.51140, 0.647, -0.631, 0.714
16, 119.12, 0.49465, 0.678, -0.673, 0.757
17, 125.78, 0.47755, 0.709, -0.715, 0.800
18, 132.64, 0.46023, 0.737, -0.757, 0.841
19, 139.34, 0.44276, 0.765, -0.799, 0.883
20, 146.12, 0.42528, 0.792, -0.841, 0.923


Run predictions with the initial parameters and the trained parameters.

In [7]:
Pred0 = higp.gpr_prediction(data_train=train_x,
                            label_train=train_y,
                            data_prediction=test_x,
                            kernel_type=higp.GaussianKernel,
                            gp_params=np.hstack((0.0, 0.0, 0.0)))

Pred = higp.gpr_prediction(data_train=train_x,
                           label_train=train_y,
                           data_prediction=test_x,
                           kernel_type=higp.GaussianKernel,
                           gp_params=model.get_params())

Finally, let us check the root mean squared error (RMSE) of the predition.

In [8]:
rmse0 = np.linalg.norm(Pred0.prediction_mean - test_y) / np.sqrt(float(n_test))
rmse = np.linalg.norm(Pred.prediction_mean - test_y) / np.sqrt(float(n_test))
print("RMSE (before training): %g, RMSE (after training): %g\n" % (rmse0, rmse))

RMSE (before training): 0.0093166, RMSE (after training): 0.00428992

